In [ ]:
!python --version

Python 3.10.12


In [ ]:
# !pip install tiktoken
# https://github.com/openai/tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.8 MB/s eta 0:00:00


In [ ]:
import re
import glob
import json
import unicodedata
import pandas as pd

In [ ]:
!mkdir text_list_csv

In [ ]:
# txt -> csv
file_list = glob.glob("*.txt")
row_data = []

for file in file_list:
  file_name = "text_list_csv/" + file[:13] + ".csv"
  with open(file)as f:
    text = f.read()
    text = text.replace('\n', '').replace(' ', '')
    text = re.sub('【\d{4}】', '', text)
  # text = "(１２３４)から（１）、（２）を抽出"

  c1 = ""
  c2 = ""
  text = unicodedata.normalize('NFKC', text).replace('\u309a', '').replace('\u2014', '')
  if re.search('(?<=【発明を実施するための形態】)(.*?)(?=【\w+】)', text):
    c1 = re.search('(?<=【発明を実施するための形態】)(.*?)(?=【\w+】)', text).group()
  if re.search('(?<=【実施例】)(.*?)(?=【\w+】)', text):
    c2 = re.search('(?<=【実施例】)(.*?)(?=【\w+】)', text).group()
  text = c1 + c2
  b_text =  [x + '\n\n' for x in text[0::1200]]
  with open('sample999.txt', 'w', encoding='shift-jis')as f:
    f.write(text)
  sentence_list = []
  for s in text.split('。'):
    if len(s) > 256:
      for pos in range(0, len(s), 256):
        sentence_list.append(s[pos:pos + 256])
    else:
      sentence_list.append(s)
  dic_for_df = {}
  dic_for_df['text'] = sentence_list
  df = pd.DataFrame(dic_for_df)
  df.to_csv(file_name, index=False, encoding='shift-jis')

In [ ]:
!rm -rf csvs_for_merge
!mkdir csvs_for_merge

In [ ]:
# csv結合
file_lst = glob.glob('csvs_for_merge/*.csv')
data_lst = []
for file in file_lst:
  data_lst.append(pd.read_csv(file, encoding='shift-jis', header=None).drop(0))
df = pd.concat(data_lst, axis=0, sort=True)
csv_name = 'hsec_total_uq.csv'
df.to_csv(csv_name, encoding='shift-jis', index=None, header=None)

In [ ]:
file = pd.read_csv(csv_name, encoding='shift-jis', header=None)
dict_lst = []
for col in file.values:
  d = {}
  text = col[0]
  d['text'] = text
  wordlist = []
  for w in col[1:]:
    if not pd.isna(w):
      wordlist.append(w)

  elist = []
  for e in wordlist:
    worddict = {}
    worddict['name'] = e
    end_pos = text.find(e) + len(e)
    worddict['span'] = [text.find(e), end_pos]
    id = "技術語"
    worddict['type'] = id
    elist.append(worddict)
  d['entities'] = elist
  dict_lst.append(d)
print(dict_lst)

with open('hsec_unique_tag.json', 'w') as f:
  json.dump(dict_lst, f, indent=2, ensure_ascii=False)

[{'text': '図３に示すように、充電ケーブル装置１０は、例えばモータを走行駆動源とする電気自動車やプラグインハイブリッド自動車等の車両５０に搭載された電池に充電を行う際に、給電源である充電スタンド４０から車両５０へ電力を供給するためのものである。', 'entities': [{'name': '充電ケーブル装置', 'span': [9, 17], 'type': '技術語'}, {'name': 'モータ', 'span': [24, 27], 'type': '技術語'}, {'name': '走行駆動源', 'span': [28, 33], 'type': '技術語'}, {'name': '電気自動車', 'span': [36, 41], 'type': '技術語'}, {'name': 'プラグインハイブリッド自動車', 'span': [42, 56], 'type': '技術語'}, {'name': '車両', 'span': [58, 60], 'type': '技術語'}, {'name': '電池', 'span': [68, 70], 'type': '技術語'}, {'name': '充電スタンド', 'span': [85, 91], 'type': '技術語'}]}, {'text': '充電ケーブル装置１０は、図１および図２に示すように、充電スタンド４０に接続されるケーブル１１と、ケーブル１１の先端に設けられた充電ガン１２と、他の充電ケーブル装置１０が接続される接続部１３と、を備えている。', 'entities': [{'name': '充電ケーブル装置', 'span': [0, 8], 'type': '技術語'}, {'name': '充電スタンド', 'span': [26, 32], 'type': '技術語'}, {'name': 'ケーブル', 'span': [2, 6], 'type': '技術語'}, {'name': '充電ガン', 'span': [63, 67], 'type': '技術語'}]}, {'text': 'ケーブル１１は、一端には充電ガン１２が設けられ、他端にはコンセントプラグ１４が設けられる。', 'entities': [{'name': 'ケーブル', 'span': [0

In [ ]:
# 文字位置の観測
# 部分一致語、同一単語による位置の重複を避けるため、文字列の検索位置を適宜更新する。
# プログラムの仕様上、位置情報を知りたい語句がリスト内に出現順に並んでいる必要がある。
dataset = []
uq_words = ['偏光子保護フィルム', 'ポリエステルフィルム', 'フィルム']  #　テキスト内の固有表現

sample = {}
text = "本発明の偏光子保護フィルムは、ポリエステルフィルムであり、フィルムの流れ方向又は幅方向に対する熱収縮率が最大となる方向の傾きの絶対値が15度以下であることが好ましい" #サンプルのテキスト
sample['text'] = text

e_item = [] #entitiesに収容される要素のひとまとまり
next_search_spos = 0
for entity in uq_words:
  e_info = {}
  e_info['name'] = entity
  end_pos = len(text[:next_search_spos]) + text[next_search_spos:].find(entity) + len(entity)
  e_info['span'] = [len(text[:next_search_spos]) + text[next_search_spos:].find(entity), end_pos]
  id = '技術語'
  e_info['type'] = id
  e_item.append(e_info)
  next_search_spos = end_pos
sample['entities'] = e_item
dataset.append(sample)

print(text[4:13])
print(dataset)

偏光子保護フィルム
[{'text': '本発明の偏光子保護フィルムは、ポリエステルフィルムであり、フィルムの流れ方向又は幅方向に対する熱収縮率が最大となる方向の傾きの絶対値が15度以下であることが好ましい', 'entities': [{'name': '偏光子保護フィルム', 'span': [4, 13], 'type': '技術語'}, {'name': 'ポリエステルフィルム', 'span': [15, 25], 'type': '技術語'}, {'name': 'フィルム', 'span': [29, 33], 'type': '技術語'}]}]


In [ ]:
word_list = ["カモシカ", "シカ", "アシカ"]
text = "カモシカもシカも確かに鹿だが、アシカは確か鹿ではない"

#変数を用意して繰り返しの都度、検索する範囲を変更する
next_search_spos = 0
for word in word_list:
  start_pos = len(text[:next_search_spos]) + text[next_search_spos:].find(word) # 開始位置
  end_pos =  len(text[:next_search_spos]) + text[next_search_spos:].find(word) + len(word) # 終了位置
  print(word, start_pos, end_pos) # (リスト要素, 開始位置, 終了位置)
  next_search_spos = end_pos

カモシカ 0 4
シカ 5 7
アシカ 15 18


In [ ]:
import tiktoken  # type: ignore[import]


def to_str(b: bytes) -> str:
    return b.decode()


def join_bytes(bs: list[bytes]) -> bytes:
    return b"".join(bs)


def join_indices(indices: list[int]) -> str:
    return "-".join(str(i) for i in indices)


def join_ids(ids: list[int]) -> str:
    return ", ".join(str(id) for id in ids)


def print_if_decoded(b: bytes, indices: str, ids: str) -> None:
    try:
        s = to_str(b)
    except UnicodeDecodeError as err:
        raise err
    else:
        print(indices, f"{s!r} ({ids})")


# prompt = "お誕生日おめでとう"
# prompt = """あなたはどんな質問にも親切に答えますが、語尾には必ず「ウマ」を付けて喋ります。"""
prompt = """あなたは架空のニュースアプリであるNewsPickerのイメージキャラクターのumaです。
あなたはどんな質問にも親切に答えますが、語尾には必ず「ウマ」を付けて喋ります。"""

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")


token_ids = encoding.encode(prompt)
byte_tokens = encoding.decode_tokens_bytes(token_ids)

rest_tokens: list[tuple[int, bytes, int]] = []
for i, (token_id, token) in enumerate(zip(token_ids, byte_tokens)):
    try:
        print_if_decoded(token, str(i), str(token_id))
    except UnicodeDecodeError:
        try:
            joined_token = join_bytes([p[1] for p in rest_tokens] + [token])
            joined_indices = join_indices([p[0] for p in rest_tokens] + [i])
            token_ids = join_ids([p[2] for p in rest_tokens] + [token_id])
            print_if_decoded(joined_token, joined_indices, token_ids)
        except UnicodeDecodeError:
            rest_tokens.append((i, token, token_id))
        else:
            rest_tokens.clear()